<a href="https://colab.research.google.com/github/ankit-rathi/DE-with-AWS/blob/main/Try_AWS_Glue_with_Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 3.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os

import pandas as pd

project_path = '/content/drive/My Drive/Personal'
os.chdir(project_path)

Mounted at /content/drive


In [3]:
import boto3
import json
import time

aws_keys_df = pd.read_csv('aws-rootkey.csv')

# Replace with your actual AWS Access Key ID and Secret Access Key
AWS_ACCESS_KEY_ID = aws_keys_df['Access_key_ID'][0]
AWS_SECRET_ACCESS_KEY = aws_keys_df['Secret_access_key'][0]
REGION_NAME = aws_keys_df['Region'][0]

# Initialize a session using your credentials
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME
)

# Create clients for S3, Glue, and IAM
s3_client = session.client('s3')
glue_client = session.client('glue')
iam_client = session.client('iam')


In [4]:
# Define the bucket name
bucket_name = 'my-glue-bucket-rathakt'  # Replace with a unique name

# Create the S3 bucket
try:
    s3_client.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': REGION_NAME}
    )
    print(f"S3 bucket '{bucket_name}' created successfully.")
except ClientError as e:
    print(f"Error creating S3 bucket: {e}")


S3 bucket 'my-glue-bucket-rathakt' created successfully.


In [5]:
import pandas as pd

# Create a sample DataFrame and save it as CSV
df = pd.DataFrame({
    'id': [1, 2, 3],
    'value': ['a', 'b', 'c']
})
csv_file_path = '/tmp/sample_data.csv'
df.to_csv(csv_file_path, index=False)

# Upload CSV to S3
s3_client.upload_file(csv_file_path, bucket_name, 'sample_data/sample_data.csv')
print("CSV file uploaded to S3.")

# Create a simple ETL script
etl_script = """
import sys
import boto3
import pandas as pd
from io import StringIO

def transform_data():
    s3_client = boto3.client('s3')
    bucket = '""" + bucket_name + """'
    key = 'sample_data/sample_data.csv'

    obj = s3_client.get_object(Bucket=bucket, Key=key)
    df = pd.read_csv(obj['Body'])

    # Example transformation: Convert 'value' column to uppercase
    df['value'] = df['value'].str.upper()

    # Save the transformed data to a new file
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    s3_client.put_object(Bucket=bucket, Key='sample_data/transformed_data.csv', Body=csv_buffer.getvalue())

if __name__ == "__main__":
    transform_data()
"""

# Save the ETL script to a file and upload it to S3
script_file_path = '/tmp/etl_script.py'
with open(script_file_path, 'w') as file:
    file.write(etl_script)

s3_client.upload_file(script_file_path, bucket_name, 'scripts/etl_script.py')
print("ETL script uploaded to S3.")


CSV file uploaded to S3.
ETL script uploaded to S3.


In [6]:
# Define the role name
role_name = 'GlueToS3Role'

# Trust relationship policy document
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "glue.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Create the IAM role
try:
    role = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description='IAM role for AWS Glue to access S3'
    )
    role_arn = role['Role']['Arn']
    print(f"Created IAM role with ARN: {role_arn}")
except ClientError as e:
    if 'EntityAlreadyExistsException' in str(e):
        role_arn = f'arn:aws:iam::{session.client("sts").get_caller_identity()["Account"]}:role/{role_name}'
        print(f"IAM role '{role_name}' already exists. ARN: {role_arn}")
    else:
        print(f"Error creating IAM role: {e}")

# Attach a policy to the role that grants access to the S3 bucket
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": ["s3:GetObject", "s3:PutObject"],
            "Resource": f"arn:aws:s3:::{bucket_name}/*"
        }
    ]
}

# Attach the policy to the role
iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName='GlueS3AccessPolicy',
    PolicyDocument=json.dumps(policy)
)
print("Attached policy to IAM role.")


Created IAM role with ARN: arn:aws:iam::419441991443:role/GlueToS3Role
Attached policy to IAM role.


In [7]:
# Define the crawler name and S3 target
crawler_name = 'my_glue_crawler'

# Create the Glue crawler
try:
    glue_client.create_crawler(
        Name=crawler_name,
        Role=role_arn,
        DatabaseName='my_glue_database',
        Targets={'S3Targets': [{'Path': f's3://{bucket_name}/sample_data/'}]},
        TablePrefix='my_'
    )
    print(f"Crawler '{crawler_name}' created successfully.")
except ClientError as e:
    if 'AlreadyExistsException' in str(e):
        print(f"Crawler '{crawler_name}' already exists.")
    else:
        print(f"Error creating Glue crawler: {e}")


Crawler 'my_glue_crawler' created successfully.


In [8]:
# Start the crawler
glue_client.start_crawler(Name=crawler_name)

# Check the crawler status
while True:
    status = glue_client.get_crawler(Name=crawler_name)['Crawler']['State']
    print(f"Crawler status: {status}")
    if status == 'READY':
        break
    time.sleep(10)

print("Crawler run completed. Data is now cataloged.")


Crawler status: RUNNING
Crawler status: RUNNING
Crawler status: RUNNING
Crawler status: RUNNING
Crawler status: RUNNING
Crawler status: RUNNING
Crawler status: RUNNING
Crawler status: STOPPING
Crawler status: STOPPING
Crawler status: STOPPING
Crawler status: STOPPING
Crawler status: STOPPING
Crawler status: STOPPING
Crawler status: STOPPING
Crawler status: READY
Crawler run completed. Data is now cataloged.


In [9]:
# Define the job name and script location
job_name = 'my_glue_job'
script_location = f's3://{bucket_name}/scripts/etl_script.py'

# Create the Glue job
try:
    glue_client.create_job(
        Name=job_name,
        Role=role_arn,
        ExecutionProperty={'MaxConcurrentRuns': 1},
        Command={
            'Name': 'glueetl',
            'ScriptLocation': script_location,
            'PythonVersion': '3'
        },
        DefaultArguments={
            '--TempDir': f's3://{bucket_name}/temp/',
            '--job-bookmark-option': 'job-bookmark-enable'
        },
        MaxRetries=0
    )
    print(f"Glue job '{job_name}' created successfully.")
except ClientError as e:
    if 'AlreadyExistsException' in str(e):
        print(f"Glue job '{job_name}' already exists.")
    else:
        print(f"Error creating Glue job: {e}")


Glue job 'my_glue_job' created successfully.


In [10]:
# Start the Glue job
job_run_id = glue_client.start_job_run(JobName=job_name)['JobRunId']
print(f"Started Glue job '{job_name}' with run ID: {job_run_id}")

# Check the job status
while True:
    job_status = glue_client.get_job_run(JobName=job_name, RunId=job_run_id)['JobRun']['JobRunState']
    print(f"Job status: {job_status}")
    if job_status in ['SUCCEEDED', 'FAILED', 'STOPPED']:
        break
    time.sleep(30)

print(f"Glue job '{job_name}' completed with status: {job_status}")


Started Glue job 'my_glue_job' with run ID: jr_88e440e20cc9cbf83a484065f29a967e7f60b8446a859eb75b3694a2767d2611
Job status: RUNNING
Job status: RUNNING
Job status: SUCCEEDED
Glue job 'my_glue_job' completed with status: SUCCEEDED


In [11]:
# Delete the Glue job
try:
    glue_client.delete_job(JobName=job_name)
    print(f"Deleted Glue job '{job_name}'")
except glue_client.exceptions.EntityNotFoundException:
    print(f"Glue job '{job_name}' does not exist.")

# Delete the Glue crawler
try:
    glue_client.delete_crawler(Name=crawler_name)
    print(f"Deleted Glue crawler '{crawler_name}'")
except glue_client.exceptions.EntityNotFoundException:
    print(f"Glue crawler '{crawler_name}' does not exist.")

# Delete the policy attached to the IAM role
try:
    iam_client.delete_role_policy(
        RoleName=role_name,
        PolicyName='GlueS3AccessPolicy'
    )
    print(f"Deleted policy from IAM role '{role_name}'")
except iam_client.exceptions.NoSuchEntityException:
    print(f"Policy for IAM role '{role_name}' does not exist.")

# Delete the IAM role
try:
    iam_client.delete_role(RoleName=role_name)
    print(f"Deleted IAM role '{role_name}'")
except iam_client.exceptions.NoSuchEntityException:
    print(f"IAM role '{role_name}' does not exist.")

# List and delete objects in the bucket
objects = s3_client.list_objects_v2(Bucket=bucket_name).get('Contents', [])
for obj in objects:
    s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
    print(f"Deleted object '{obj['Key']}' from S3 bucket '{bucket_name}'")

# Delete the S3 bucket
try:
    s3_client.delete_bucket(Bucket=bucket_name)
    print(f"Deleted S3 bucket '{bucket_name}'")
except ClientError as e:
    print(f"Error deleting S3 bucket '{bucket_name}': {e}")



Deleted Glue job 'my_glue_job'
Deleted Glue crawler 'my_glue_crawler'
Deleted policy from IAM role 'GlueToS3Role'
Deleted IAM role 'GlueToS3Role'
Deleted object 'sample_data/sample_data.csv' from S3 bucket 'my-glue-bucket-rathakt'
Deleted object 'sample_data/transformed_data.csv' from S3 bucket 'my-glue-bucket-rathakt'
Deleted object 'scripts/etl_script.py' from S3 bucket 'my-glue-bucket-rathakt'
Deleted S3 bucket 'my-glue-bucket-rathakt'
